In [1]:
# set up
import pandas as pd
import numpy as np
import pickle
import time 
import os
import matplotlib.pyplot as plt
import datetime
from operator import itemgetter
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pylab import *
from matplotlib.font_manager import FontProperties  
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
font = FontProperties(fname=r"simsun.ttf", size=14)  
matplotlib.rcParams['axes.unicode_minus'] =False
mpl.rcParams['font.sans-serif'] = ['SimHei']
tick_spacing = 4

# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)
    
def takeSecond(elem):
    return elem[1]


def retrieve_low_nodes(codes,dt,num):
    dl_rank = []
    for code in codes:
        if DTBS['A'][code][dt]['dl'] != 999:
            dl_rank.append((code, DTBS['A'][code][dt]['dl']))
    
    dl_rank.sort(key=takeSecond)
    low = dl_rank[0:num]
    return [i[0] for i in low]

def past_qs(days, code):
    
    for day in days:
            
        if DTBS['A'][code][day]['qs'] == 1:
            return 1
    
    return 0

def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if low_prc(code, dt, lm, fcd[1]):
                satisfy.append(code)
        
    else:
        for code in codes:
            if DTBS['A'][code][dt][fcd[0]] <= fcd[2] and DTBS['A'][code][dt][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DTBS['A'][code][dt]['dp']):
                lm = calculate_time(dt, 'p', 20)
                prc = get_prcs(code, lm)
                lp = absolute_low(prc, 0)
                rk.append((code, DTBS['A'][code][dt]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan(DTBS['A'][code][dt][idx]):
                rk.append((code, DTBS['A'][code][dt][idx]))
        else:
            if not np.isnan(DTBS['A'][code][dt][idx]):
                if DTBS['A'][code][dt][idx] != 999:
                    rk.append((code, DTBS['A'][code][dt][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]
  

def get_prcs(code, dts):
    prc = []
    for dt in dts:
        prc.append(DTBS['A'][code][dt]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DTBS['A'][code][dt]['dp']
#     if (tp <= lp):
#         print(code, tp, lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

def calculate_time(dt, d, l):
    time = []
    days = DTBS['D']['day']
    if d == 'p':
        for i in range(len(days)):
            if days[i] == dt:
                break
                
    return days[i-l+1:i+1]

past_dt = DTBS['D']['day'][-20:]

In [2]:
dt = DTBS['D']['day'][-1]
print(dt)
dt = DTBS['D']['day'][-1]
past_dt = DTBS['D']['day'][-20:]
re = retrieve_low_nodes(DTBS['A'].keys(), dt , 400)


2023-04-26


In [3]:
# save
f_save = open(DTBS_path, 'wb')
pickle.dump(DTBS, f_save)
f_save.close()


In [6]:
len(select)

36

In [5]:
dt = DTBS['D']['day'][-1]
dt = '2023-01-30'
# dt = '2023-04-12'
select = list(DTBS['A'].keys())
past_dt = DTBS['D']['day'][-40:]

# flt_cd = [['lp', 0.5], ['dp', 110, 130], ['bl',0, 7], ['cpr',-999, 50]]

flt_cd = [['dp', 100, 130]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]
flt_cd = [['dp', 100, 130], ['bl',0, 3]]
for fcd in flt_cd:
    select = select_condition(select, dt, fcd)
    
    
# rk_cd = [['lp', 10, 'a']]
rk_cd = [['dl', 600, 'a']]
rk_cd = [['ytm', 20, 'd']]
rk_cd = [['dl', 20, 'a']]
lp = rank(select, dt, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(lp)
print("当前日期：", dt)
for r in lp:
    print(r, DTBS['B'][r]['cn'], DTBS['B'][r]['sc'],  DTBS['B'][r]['sn'],  '强赎：' , past_qs(past_dt, r), "收盘价：", DTBS['A'][r][dt]['dp'],  "转股溢价率：", DTBS['A'][r][dt]['cpr'], "到期收益率：", DTBS['A'][r][dt]['ytm'],"换手率：", DTBS['A'][r][dt]['trt'], '市值',  DTBS['A'][r][dt]['bl'], '双低', DTBS['A'][r][dt]['dl'], '申万一级', DTBS['B'][r]['cat1'],'申万二级', DTBS['B'][r]['cat2'])

当前日期： 2023-01-30
128033.SZ 迪龙转债 002658.SZ 雪迪龙 强赎： 0 收盘价： 129.856 转股溢价率： 21.4735 到期收益率： -20.0555 换手率： 7.2498 市值 2.9723 双低 151.3295 申万一级 环保(2021) 申万二级 环保设备Ⅱ(2021)
110070.SH 凌钢转债 600231.SH 凌钢股份 强赎： 0 收盘价： 116.528 转股溢价率： 35.3397 到期收益率： 0.0471 换手率： 5.3038 市值 2.1705 双低 151.8677 申万一级 钢铁(2021) 申万二级 普钢(2021)
113546.SH 迪贝转债 603320.SH 迪贝电气 强赎： 0 收盘价： 124.783 转股溢价率： 27.7452 到期收益率： -1.8727 换手率： 5.3586 市值 2.2963 双低 152.5282 申万一级 电力设备(2021) 申万二级 电机Ⅱ(2021)
128076.SZ 金轮转债 002722.SZ 物产金轮 强赎： 0 收盘价： 124.6 转股溢价率： 28.3142 到期收益率： -3.5037 换手率： 12.8266 市值 2.137 双低 152.9142 申万一级 纺织服饰(2021) 申万二级 纺织制造(2021)
128021.SZ 兄弟转债 002562.SZ 兄弟科技 强赎： 0 收盘价： 125.719 转股溢价率： 27.4378 到期收益率： -18.6331 换手率： 5.4365 市值 2.6515 双低 153.1568 申万一级 基础化工(2021) 申万二级 化学制品(2021)
128071.SZ 合兴转债 002228.SZ 合兴包装 强赎： 0 收盘价： 114.491 转股溢价率： 39.3204 到期收益率： -0.4169 换手率： 2.7097 市值 2.9993 双低 153.8114 申万一级 轻工制造(2021) 申万二级 包装印刷(2021)
113526.SH 联泰转债 603797.SH 联泰环保 强赎： 0 收盘价： 128.624 转股溢价率： 27.7308 到期收益率： -8.5854 换手率： 6.7476 市值 1.9274 双低 156.3547999999999

In [3]:
dt = DTBS['D']['day'][-1]
# dt = '2023-04-12'
select = list(DTBS['A'].keys())

# flt_cd = [['lp', 0.5], ['dp', 110, 130], ['bl',0, 7], ['cpr',-999, 50]]
flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]
# flt_cd = [['dp', 100, 130]]
# flt_cd = [['dp', 100, 130], ['bl',0, 3]]
for fcd in flt_cd:
    select = select_condition(select, dt, fcd)
    
    
rk_cd = [['lp', 10, 'a']]
# rk_cd = [['dl', 600, 'a']]
# rk_cd = [['dl', 10, 'a']]
lp = rank(select, dt, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(lp)
print("当前日期：", dt)
for r in lp:
    print(r, DTBS['B'][r]['cn'], DTBS['B'][r]['sc'],  DTBS['B'][r]['sn'],  '强赎：' , past_qs(past_dt, r), "收盘价：", DTBS['A'][r][dt]['dp'],  "转股溢价率：", DTBS['A'][r][dt]['cpr'], "到期收益率：", DTBS['A'][r][dt]['ytm'],"换手率：", DTBS['A'][r][dt]['trt'], '市值',  DTBS['A'][r][dt]['bl'], '双低', DTBS['A'][r][dt]['dl'], '申万一级', DTBS['B'][r]['cat1'],'申万二级', DTBS['B'][r]['cat2'])

当前日期： 2023-04-24
110070.SH 凌钢转债 600231.SH 凌钢股份 强赎： 0 收盘价： 115.898 转股溢价率： 42.26 到期收益率： -0.09 换手率： 3.85 市值 2.171 双低 158.158 申万一级 钢铁(2021) 申万二级 普钢(2021)
123166.SZ 蒙泰转债 300876.SZ 蒙泰高新 强赎： 0 收盘价： 124.5 转股溢价率： 31.65 到期收益率： -0.45 换手率： 2.36 市值 3.0 双低 156.15 申万一级 基础化工(2021) 申万二级 化学纤维(2021)
128071.SZ 合兴转债 002228.SZ 合兴包装 强赎： 0 收盘价： 113.4 转股溢价率： 43.62 到期收益率： -0.04 换手率： 2.79 市值 2.999 双低 157.02 申万一级 轻工制造(2021) 申万二级 包装印刷(2021)
113030.SH 东风转债 601515.SH 东风股份 强赎： 0 收盘价： 118.887 转股溢价率： 37.78 到期收益率： -1.16 换手率： 4.52 市值 2.945 双低 156.667 申万一级 轻工制造(2021) 申万二级 包装印刷(2021)
113546.SH 迪贝转债 603320.SH 迪贝电气 强赎： 0 收盘价： 119.691 转股溢价率： 40.22 到期收益率： -0.37 换手率： 2.76 市值 2.297 双低 159.911 申万一级 电力设备(2021) 申万二级 电机Ⅱ(2021)
113595.SH 花王转债 603007.SH ST花王 强赎： 0 收盘价： 118.429 转股溢价率： -0.46 到期收益率： 1.46 换手率： 58.47 市值 2.708 双低 117.96900000000001 申万一级 建筑装饰(2021) 申万二级 基础建设(2021)
123061.SZ 航新转债 300424.SZ 航新科技 强赎： 0 收盘价： 123.804 转股溢价率： 40.57 到期收益率： -0.08 换手率： 7.09 市值 2.496 双低 164.374 申万一级 国防军工(2021) 申万二级 航空装备Ⅱ(2021)
128076.SZ 金轮转债 002722.SZ 

In [5]:
dt = DTBS['D']['day'][-1]
dt

'2023-04-11'

In [6]:
lm =  DTBS['D']['day'][-20:]

In [4]:
(196.96 - 197.9855) / 197.9855

-0.0051796722487252546

In [61]:

# code = '118016.SH'
# prc = []
# for dt in lm:
#     prc.append(DTBS['A'][code][dt]['dp'])
# prc.sort()    
# max_prc = max(prc)
# min_prc = min(prc)
# print(prc)
# print(max_prc, min_prc)

def get_prcs(code, dts):
    prc = []
    for dt in dts:
        prc.append(DTBS['A'][code][dt]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DTBS['A'][code][dt]['dp']
#     print(code, tp, lp, tp <= lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

In [62]:
113600.SH

SyntaxError: invalid decimal literal (3743447309.py, line 1)

In [63]:
max([1,2,3])

3

In [64]:
low_prc('123128.SZ', DTBS['D']['day'][-1], DTBS['D']['day'][-20:], 0.2)


123128.SZ 102.006 100.96039999999999 False


False

In [65]:
print(min(prc))
print(np.quantile(a=prc, q=1))

NameError: name 'prc' is not defined

In [66]:


relative_low(prc, 0.20)
absolute_low(prc, 0.20)

NameError: name 'prc' is not defined

In [25]:
lm =  DTBS['D']['day'][-22:]
dt = DTBS['D']['day'][-1]
pos = 0
low_code = []
for code in DTBS['A'].keys():
    flag = 0
    for odt in lm:
        if DTBS['A'][code][odt]['ia'] == 0:
            flag = 1
    if flag == 0:
        if low_prc(code, dt, lm, pos):
            low_code.append(code)
    

print(len(low_code))

lowlow = []
for code in low_code:
    if DTBS['A'][code][dt]['bl'] <3:
        lowlow.append(code)
        
print(len(lowlow))
print(lowlow)

87
31
['113566.SH', '123012.SZ', '123013.SZ', '123118.SZ', '128039.SZ', '113030.SH', '113574.SH', '123014.SZ', '123034.SZ', '123067.SZ', '123080.SZ', '123159.SZ', '128070.SZ', '128076.SZ', '128120.SZ', '128143.SZ', '113537.SH', '113575.SH', '123011.SZ', '123072.SZ', '127004.SZ', '128056.SZ', '128082.SZ', '113565.SH', '123018.SZ', '128021.SZ', '128025.SZ', '128053.SZ', '113573.SH', '127080.SZ', '128033.SZ']


In [49]:
print(low_code)

['110088.SH', '113631.SH', '123128.SZ', '113046.SH', '113535.SH', '113623.SH', '113641.SH', '118000.SH', '118005.SH', '118010.SH']


In [30]:
DTBS['A'][code][dt]['dp']

117.89200000000001

In [40]:
DTBS['A']['113600.SH'][DTBS['D']['day'][-1]]['dp']

119.941

In [88]:



low_prc('128105.SZ', DTBS['D']['day'][-1], DTBS['D']['day'][-20:], 0.2)



128105.SZ 106.208 106.4964 106.6056 106.4964


True

In [22]:
dt = DTBS['D']['day'][-1]
can = []
for code in DTBS['A'].keys():

    if DTBS['A'][code][dt]['ia'] == 1:
#         print(DTBS['A'][code][dt]['bl'])
        if DTBS['A'][code][dt]['bl'] <= 7:
            if DTBS['A'][code][dt]['trt'] >= 2:
                if DTBS['A'][code][dt]['trt'] <= 10:
                    if DTBS['A'][code][dt]['ytm'] >= -1:
                        can.append(code)

print(len(can))
re = retrieve_low_nodes(can, dt , 400)

26


In [ ]:
%matplotlib notebook